In [1]:
import yaml
import pathlib
import random
import panel as pn
import pandas as pd
pn.extension() 

def load_recipe(path):
    with path.open("r") as f:
        return yaml.load(f, Loader=yaml.FullLoader)
    
def spin(chosen, recipes, N_total):
    return random.sample(list(filter(lambda rec: rec not in chosen, recipes)), k = N_total-len(chosen))


def make_callback(recipes, bgroup, N_total=5):
    def callback(*events):
        chosen = bgroup.value
        suggestions = spin(chosen, recipes, N_total)
        bgroup.options = chosen + suggestions

    return callback

out = {"menu":""}


def make_printer(recipes, bgroup, static_text):
    def print_menu(*events):
        chosen = bgroup.value
        s = ""
        for value in chosen:
            fname = recipes[value]["path"]
            menu_item = f"""
# {value} (dag):
    - {fname}:
        amount: 4
        unit: plates
"""
            s += menu_item
        out["menu"] = s
        static_text.object=s
    
        df = pd.DataFrame([s])
        df.to_clipboard(index=False,header=False)
    return print_menu

blacklist = [
    "Chokoladetærte med saltkaramel",
    "Peberkager",
    "Coleslaw",
    "Dilddressing",
    'Pitabrød',
    'Knuste Kartofler',
    'Flødekartofler',
    'Blomkålssalat med æbler og mandler',
    'Kartoffelmos',
    'Broccolisalat',
    'Anders Ands kanelsnegle',
    'Hvidkålssalat',
    'Rucolasalat med bagte tomater',
    'Raita',
    'Pirogger med oksekød',
    'Pita brød',
    'Mango lassi',
    'TikTok baked oats',
    'Spinatsalat med feta og granatæble',
    'Spinat Pandekager',
    'Pizzadej',
    'Raw muslibar',
    'Mettes Æblekage',
    'Rugboller',
    'Naan',
    'Bedstemor ands chokoladekage',
    "Verdens bedste burger",
    "Kyllinge Nuggets med Corn Flakes",
    "Calzoneboller",
    "Bananbrød",
    "standard",
]
             
paths = pathlib.Path("recipes").glob("*.yml")
recipes = {p.stem:load_recipe(p) for p in paths}
recipes = {rec["navn"]: {**rec, "path":p} for p,rec in recipes.items() if not rec["navn"] in blacklist}


In [2]:
chosen = [
]
N_total=6
checkbutton_group = pn.widgets.CheckButtonGroup(
    name = 'Check Button Group',
    value = chosen,
    options = chosen + spin(chosen, recipes, N_total=N_total),
    orientation = "vertical",
    button_type="primary",
    width=400,
)


spin_button = pn.widgets.Button(name="Spin")
menu_button = pn.widgets.Button(name="Lav Menuen")
static_text = pn.pane.Markdown('', height=400, width=400)

spin_button.on_click(make_callback(recipes, checkbutton_group, N_total=N_total))
menu_button.on_click(make_printer(recipes, checkbutton_group, static_text))

autocomplete = pn.widgets.AutocompleteInput(
    name='Find opskrift',
    options=list(recipes.keys()),
    placeholder='Skriv starten af navnet',
    case_sensitive=False,
    min_characters=1,
)


ui = pn.Row(autocomplete, height=200)


def make_callback_2(checkbutton_group, autocomplete):
    
    def callback(event, *args):
        if not event.type == "changed":
            print(event.type)
            return
        
        if autocomplete.value == "":
            return
        
        selected = checkbutton_group.value + [event.new]
        n_selected = len(selected)
        options = selected + checkbutton_group.options[n_selected:]
        
        checkbutton_group.options = options
        checkbutton_group.value = selected
        
        autocomplete.value = ""

    return callback

watcher = autocomplete.param.watch(make_callback_2(checkbutton_group, autocomplete), 'value', onlychanged=True)


import ipywidgets as widgets
from IPython.display import display

# Some arbitrary options

# {'Kyllingesalat med æble': 'Kyllingesalat med æble bacon cremefraiche 18 dild groft brød honning karry kyllingebryst m skind mayonnaise olivenolie purløg rucola rødløg salat smør æble æblecidereddike',
# 'Bønnegryde med linser': 'Bønnegryde med linse ...
names_to_ingredients = {k: k+" "+" ".join(v["ingredienser"]) for k,v in recipes.items()}
OPTIONS = [k for k in names_to_ingredients]


# Create a text input widget
text = widgets.Text()
# Create a Select widget
select = widgets.Select(options=OPTIONS)

# Function to update select options based on text input
def update_select_options(*args):
    typed_text = text.value
    updated_options = [option for option, desc in names_to_ingredients.items() if typed_text.lower() in desc.lower() + option.lower()]
    # Unobserve the event temporarily
    select.unobserve(on_selection_change, names='value')
    select.options = updated_options
    # Observe the event again
    select.observe(on_selection_change, names='value')    

debug = {"events" : []}
# Function to handle selection
def on_selection_change(change):
    # Here, change is a dictionary holding 'name', 'old', 'new', 'owner', 'type'
    # 'new' will be the value of the new selection
    debug["events"].append(change)
    to_add = change['new']
    
    selected = checkbutton_group.value + [to_add]
    n_selected = len(selected)
    options = selected + checkbutton_group.options[n_selected:]

    checkbutton_group.options = options
    checkbutton_group.value = selected

    # Your other stuff here, and then:
    text.value = ""
    
    
# Attach the update function to text value changes
text.observe(update_select_options, 'value')
select.observe(on_selection_change, names='value')

ui = pn.Column(
    pn.Row(pn.Column(checkbutton_group, static_text),
    pn.Column(spin_button, menu_button, text, select, height=600)),    
)

ui.objects[0][1][2].param.watch(update_select_options, 'object')
ui.objects[0][1][3].param.watch(on_selection_change, 'object')


ui.servable()


Column
    [0] Row
        [0] Column
            [0] CheckButtonGroup(button_type='primary', name='Check Button Group', options=['Bøf Stroganoff', ...], orientation='vertical', width=400)
            [1] Markdown(str, height=400, sizing_mode='fixed', width=400)
        [1] Column(height=600)
            [0] Button(name='Spin')
            [1] Button(name='Lav Menuen')
            [2] IPyWidget(Text)
            [3] IPyWidget(Select)